# 🎨 MyVLM-Art 训练 (Clean Version)

⚠️ **要求**: GPU 运行时 (T4 或更高)

**运行顺序**:
1. 运行 Cell 1 (安装依赖)
2. 运行 Cell 2 (重启 - 会自动重启)
3. **重启后** 从 Cell 3 开始运行

## Cell 1: 安装依赖

⚠️ 运行完后必须重启 Runtime！

In [ ]:
%%capture
# 完全卸载旧版本
!pip uninstall -y bitsandbytes transformers accelerate -q

# 安装固定版本
!pip install transformers==4.36.2 --no-cache-dir -q
!pip install accelerate==0.25.0 --no-cache-dir -q
!pip install bitsandbytes==0.41.3 --no-cache-dir -q
!pip install peft>=0.2.0 -q
!pip install open-clip-torch -q
!pip install timm>=0.9.16 -q
!pip install sentence-transformers>=2.3.1 -q
!pip install omegaconf>=2.3.0 -q
!pip install pyrallis>=0.3.1 -q
!pip install loguru>=0.7.0 -q
!pip install wandb>=0.16.0 -q

In [ ]:
print("✅ 依赖安装完成")
print("⚠️ 现在运行下一个 Cell 重启 Runtime")

## Cell 2: 重启 Runtime

⚠️ **运行后自动重启，重启后跳过 Cell 1-2，直接从 Cell 3 开始**

In [ ]:
import os
os.kill(os.getpid(), 9)

---
# ⬇️⬇️⬇️ 重启后从这里开始 ⬇️⬇️⬇️
---

## Cell 3: 验证环境 + Clone 代码

In [ ]:
# 验证版本
import torch
import transformers
print(f"PyTorch: {torch.__version__}")
print(f"Transformers: {transformers.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 检查版本
assert transformers.__version__.startswith("4.36"), f"transformers 版本应为 4.36.x，当前: {transformers.__version__}"
print("\n✅ 环境验证通过")

## Cell 5b: 只测试 Stage B（从 checkpoint 恢复）

In [ ]:
# ==================== 只测试 Stage B（从 checkpoint 恢复） ====================
# ⚠️ 前提：之前已经运行过 Cell 5 完成了训练，checkpoint 在本地
# ⚠️ 如果 runtime 已断开重连，需要先从 Google Drive 恢复文件

import sys
import os
from pathlib import Path
from glob import glob

os.chdir("/content/MyVLM_art")
sys.path.insert(0, "/content/MyVLM_art")

# 清除模块缓存
modules_to_clear = [m for m in list(sys.modules.keys()) 
                    if any(x in m for x in ['vlms', 'myvlm', 'concept_graph', 'configs'])]
for m in modules_to_clear:
    del sys.modules[m]

import torch
from myvlm.common import VLMType, PersonalizationTask, VLM_TO_LAYER, MyVLMLayerMode
from configs.myvlm_art_config import MyVLMArtConfig
from vlms.llava_wrapper import LLaVAWrapper
from myvlm.myllava import MyLLaVA
from concept_graph.concept_embeddings.trainer import MultiTokenEmbeddingTrainer
from myvlm.utils import parent_module, brackets_to_periods
import wandb
from tqdm import tqdm

# ========== 自动查找最新的 checkpoint ==========
checkpoint_dir = Path("/content/MyVLM_art/outputs/dataset/seed_42")
checkpoints = sorted(checkpoint_dir.glob("checkpoints_*.pt"))

if checkpoints:
    CHECKPOINT_PATH = str(checkpoints[-1])  # 最新的
    print(f"📂 找到 {len(checkpoints)} 个 checkpoint")
    print(f"📥 使用最新的: {CHECKPOINT_PATH}")
else:
    # 手动指定路径
    CHECKPOINT_PATH = "outputs/dataset/seed_42/checkpoints_dataset_seed_42_20251206_205955.pt"
    print(f"⚠️ 未找到 checkpoint，使用指定路径: {CHECKPOINT_PATH}")

cfg = MyVLMArtConfig(
    concept_name="dataset",
    concept_identifier="test",
    vlm_type=VLMType.LLAVA,
    personalization_task=PersonalizationTask.CAPTIONING,
    output_root=Path("./outputs"),
    data_root=Path("./data"),
    optimization_steps=1,
    learning_rate=1.0,
    batch_size=1,
    reg_lambda=0.0,
    device='cuda',
    torch_dtype=torch.bfloat16,
    threshold=0.75,
    max_tokens_per_concept=1,
    max_concepts_per_sample=1,
    backoff_delta=0.05,
    val_subset_n=3,
    max_reason_tokens=64,
    use_wandb=True,
    wandb_project="myvlm-art-test",
    wandb_run_name="stage-b-only-test",
)

# ========== 加载模型 ==========
print("🔄 加载 LLaVA 模型...")
vlm = LLaVAWrapper(device=cfg.device, torch_dtype=cfg.torch_dtype)

print("🔧 初始化 MyLLaVA...")
myvlm = MyLLaVA(vlm, layer=VLM_TO_LAYER[cfg.vlm_type], concept_name=cfg.concept_name, cfg=cfg)

# ========== 初始化 Trainer（会创建数据集和注入层） ==========
print("📦 初始化 Trainer...")
trainer = MultiTokenEmbeddingTrainer(cfg=cfg, myvlm=myvlm, dataset_builder=None)

# ========== 加载 checkpoint 到注入层 ==========
print(f"📥 加载 checkpoint: {CHECKPOINT_PATH}")
ckpt = torch.load(CHECKPOINT_PATH, map_location="cpu")
# 获取最新的 checkpoint 状态
latest_step = max(ckpt.keys())
state = ckpt[latest_step]
print(f"  - 加载 step {latest_step} 的状态")
print(f"  - keys shape: {state.get('keys', torch.tensor([])).shape}")
print(f"  - values shape: {state.get('values', torch.tensor([])).shape}")

# 注入到层中
layer = trainer._injected_layer
print(f"  - _injected_layer: {layer}")
print(f"  - 'values' in state: {'values' in state}")
if layer is not None and 'values' in state:
    layer.values = torch.nn.Parameter(state['values'].to(cfg.device, cfg.torch_dtype))
    if 'keys' in state and state['keys'].numel() > 0:
        layer.keys = state['keys'].to(cfg.device, cfg.torch_dtype)
    print("✅ Checkpoint 已加载到模型")
    print(f"  - layer.values shape: {layer.values.shape if layer.values is not None else 'None'}")
else:
    print("❌ 无法加载 checkpoint")
    print(f"  - layer is None: {layer is None}")
    print(f"  - 'values' in state: {'values' in state}")

# ========== 只运行 Stage B ==========
print("\n🚀 开始 Stage B 测试...")

# 初始化 wandb
wandb.init(
    project=cfg.wandb_project,
    name=cfg.wandb_run_name,
    config={"test_type": "stage_b_only", "checkpoint": CHECKPOINT_PATH},
)

# 设置训练模式
setattr(eval(f"myvlm.vlm.{myvlm.layer}"), "training", True)
setattr(eval(f"myvlm.vlm.{myvlm.layer}"), "mode", MyVLMLayerMode.TRAIN)

# 创建 optimizer
optimizer = torch.optim.AdamW(myvlm.vlm.model.parameters(), lr=cfg.learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ConstantLR(optimizer, cfg.learning_rate)

# 运行 Stage B
dl_b = trainer._train_loaders["B"]
print(f"[Stage B] 数据集大小: {len(dl_b.dataset)}")

global_step = 0
for batch_idx, batch in enumerate(tqdm(dl_b, desc="Stage B")):
    if batch_idx >= 5:  # 只测试 5 个 batch
        break
    
    # ========== 诊断：检查 batch 内容 ==========
    labels = batch.get("labels")
    input_ids = batch.get("input_ids")
    
    if labels is not None:
        # 统计有效 token（非 -100）
        valid_tokens = (labels != -100).sum().item()
        total_tokens = labels.numel()
        print(f"\n  [Batch {batch_idx}] 诊断:")
        print(f"    - labels shape: {labels.shape}")
        print(f"    - 有效 token 数: {valid_tokens} / {total_tokens}")
        print(f"    - labels 前 20 个值: {labels[0, :20].tolist()}")
        print(f"    - labels 后 20 个值: {labels[0, -20:].tolist()}")
        
        if valid_tokens == 0:
            print(f"    ⚠️ 所有 labels 都是 -100！这就是 loss=0 的原因！")
    
    with torch.cuda.amp.autocast():
        outputs = myvlm.vlm.model(**batch)
    
    loss = outputs.loss
    raw_loss = loss.item() if hasattr(loss, 'item') else float(loss)
    
    # ========== 深度诊断 ==========
    print(f"    - loss = {raw_loss:.6f} (raw: {raw_loss:.2e})")
    print(f"    - loss type: {type(loss)}, dtype: {loss.dtype if hasattr(loss, 'dtype') else 'N/A'}")
    
    # 检查 logits
    if hasattr(outputs, 'logits') and outputs.logits is not None:
        logits = outputs.logits
        print(f"    - logits shape: {logits.shape}")
        print(f"    - logits 范围: [{logits.min().item():.4f}, {logits.max().item():.4f}]")
        
        # 检查 labels 是否在 outputs 中被扩展
        if hasattr(outputs, 'labels') and outputs.labels is not None:
            print(f"    - outputs.labels shape: {outputs.labels.shape}")
        
        # 注意：LLaVA 会把图像展开成 patch tokens，所以 logits 比 labels 长
        # 不能直接手动计算 loss，需要依赖模型内部的 loss 计算
        print(f"    ⚠️ logits ({logits.shape[1]}) vs labels ({labels.shape[1]}) 长度不匹配！")
        print(f"    这是因为 LLaVA 把图像展开成了 {logits.shape[1] - labels.shape[1]} 个 patch tokens")
    else:
        print(f"    - ⚠️ outputs 没有 logits 属性")
    
    # wandb 日志
    wandb.log({
        "stage": "B",
        "batch": batch_idx,
        "train/loss": raw_loss,
        "train/loss_log10": torch.log10(torch.tensor(max(raw_loss, 1e-10))).item(),
    }, step=global_step)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)
    global_step += 1

wandb.finish()
print("\n✅ Stage B 测试完成!")

## Cell 5c: 快速训练测试（修复后）

In [ ]:
# ==================== 中等规模训练测试 ====================
# ⚠️ 如果之前运行过其他测试，需要重新加载模型
# 预计时间：5-10 分钟

import sys
import os
os.chdir("/content/MyVLM_art")

# 清除所有相关模块缓存
modules_to_clear = [m for m in list(sys.modules.keys()) 
                    if any(x in m for x in ['vlms', 'myvlm', 'concept_graph', 'configs'])]
for m in modules_to_clear:
    del sys.modules[m]
print(f"✅ 清除 {len(modules_to_clear)} 个模块缓存")

# ========== 应用兼容性 Patch ==========
# Patch 1: 量化配置冲突
import vlms.llava.model.builder as builder_module
_orig_load = builder_module.load_pretrained_model
def _patched_load(*args, **kwargs):
    if kwargs.get('load_in_4bit') and kwargs.get('quantization_config'):
        kwargs.pop('load_in_4bit', None)
    return _orig_load(*args, **kwargs)
builder_module.load_pretrained_model = _patched_load

# Patch 2: cache_position 参数
from vlms.llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
_orig_forward = LlavaLlamaForCausalLM.forward
def _patched_forward(self, *args, **kwargs):
    kwargs.pop('cache_position', None)
    return _orig_forward(self, *args, **kwargs)
LlavaLlamaForCausalLM.forward = _patched_forward
print("✅ 兼容性 Patch 已应用")

# 重新导入所有必要的模块
from pathlib import Path
import torch
from myvlm.common import VLMType, PersonalizationTask, VLM_TO_LAYER
from configs.myvlm_art_config import MyVLMArtConfig
from vlms.llava_wrapper import LLaVAWrapper
from myvlm.myllava import MyLLaVA
from concept_graph.concept_embeddings.trainer import MultiTokenEmbeddingTrainer

# 配置（测试版）
cfg = MyVLMArtConfig(
    concept_name="dataset",
    concept_identifier="test",
    vlm_type=VLMType.LLAVA,
    personalization_task=PersonalizationTask.CAPTIONING,
    output_root=Path("./outputs"),
    data_root=Path("./data"),
    optimization_steps=3,         # Stage A + Stage B 都是 3 个 epoch
    learning_rate=1.0,
    batch_size=1,                 # batch_size=1（Stage B 加权 loss 需要更多显存）
    reg_lambda=0.0,
    device='cuda',
    torch_dtype=torch.bfloat16,
    threshold=0.75,
    max_tokens_per_concept=2,     # 每个概念 2 个 token
    max_concepts_per_sample=2,    # 每个样本 2 个概念
    backoff_delta=0.05,
    val_subset_n=5,               # 验证 5 个样本
    max_reason_tokens=64,
    use_wandb=True,
    wandb_project="myvlm-art-test",
    wandb_run_name="bigger-test",
)

# 重新加载模型
print("🔄 重新加载 LLaVA 模型...")
vlm = LLaVAWrapper(device=cfg.device, torch_dtype=cfg.torch_dtype)

print("🔧 初始化 MyLLaVA...")
myvlm = MyLLaVA(vlm, layer=VLM_TO_LAYER[cfg.vlm_type], concept_name=cfg.concept_name, cfg=cfg)

print("📦 创建 Trainer...")
trainer = MultiTokenEmbeddingTrainer(cfg=cfg, myvlm=myvlm, dataset_builder=None)

# 运行完整训练（Stage A + Stage B）
print("🚀 开始训练测试...")
print(f"   Stage A: {cfg.optimization_steps} epochs × 86 batches")
print(f"   Stage B: {cfg.optimization_steps} epochs × 86 batches")
checkpoints = trainer.train()

# 保存
from datetime import datetime
from zoneinfo import ZoneInfo
out_dir = cfg.output_root / cfg.concept_name / f"seed_{cfg.seed}"
out_dir.mkdir(parents=True, exist_ok=True)
ts = datetime.now(ZoneInfo('America/New_York')).strftime("%Y%m%d_%H%M%S")
save_path = out_dir / f"checkpoints_bigger_test_{ts}.pt"
torch.save(checkpoints, save_path)

print(f"\n✅ 训练测试完成!")
print(f"💾 检查点: {save_path}")
print(f"📊 查看 wandb: https://wandb.ai")

In [ ]:
# Clone 代码
import os
import shutil

# TODO: 替换为你的仓库地址
REPO_URL = "https://github.com/YOUR_USERNAME/MyVLM_art.git"
BRANCH = "log-for-colab"

if os.path.exists("/content/MyVLM_art"):
    shutil.rmtree("/content/MyVLM_art")
    print("🗑️ 删除旧代码")

!git clone -b {BRANCH} {REPO_URL}
os.chdir("/content/MyVLM_art")
print(f"\n📁 当前目录: {os.getcwd()}")
!git log --oneline -3

## Cell 4: Wandb 登录 (可选)

In [ ]:
import wandb
wandb.login()
print("✅ Wandb 登录成功")

## Cell 5: 运行训练 🚀

In [ ]:
# ==================== 完整训练代码 ====================
import sys
import os
os.chdir("/content/MyVLM_art")
sys.path.insert(0, "/content/MyVLM_art")

# 导入并配置
from pathlib import Path
import torch
from myvlm.common import VLMType, PersonalizationTask, VLM_TO_LAYER
from configs.myvlm_art_config import MyVLMArtConfig
from vlms.llava_wrapper import LLaVAWrapper
from myvlm.myllava import MyLLaVA
from concept_graph.concept_embeddings.trainer import MultiTokenEmbeddingTrainer

# 配置参数
cfg = MyVLMArtConfig(
    concept_name="dataset",
    concept_identifier="test",
    vlm_type=VLMType.LLAVA,
    personalization_task=PersonalizationTask.CAPTIONING,
    output_root=Path("./outputs"),
    data_root=Path("./data"),
    optimization_steps=1,         # 快速测试，正式训练改为 100
    learning_rate=1.0,
    batch_size=1,                 # T4 建议 1-2
    reg_lambda=0.0,
    device='cuda',
    torch_dtype=torch.bfloat16,
    threshold=0.75,
    max_tokens_per_concept=1,
    max_concepts_per_sample=1,
    backoff_delta=0.05,
    val_subset_n=3,
    max_reason_tokens=64,
    # Wandb (设为 False 禁用)
    use_wandb=True,
    wandb_project="myvlm-art-test",
    wandb_run_name="quick-test",
)

# 训练
print("🔄 加载 LLaVA 模型...")
vlm = LLaVAWrapper(device=cfg.device, torch_dtype=cfg.torch_dtype)

print("🔧 初始化 MyLLaVA...")
myvlm = MyLLaVA(vlm, layer=VLM_TO_LAYER[cfg.vlm_type], concept_name=cfg.concept_name, cfg=cfg)

print("🚀 开始训练...")
trainer = MultiTokenEmbeddingTrainer(cfg=cfg, myvlm=myvlm, dataset_builder=None)
checkpoints = trainer.train()

# 保存
from datetime import datetime
from zoneinfo import ZoneInfo
out_dir = cfg.output_root / cfg.concept_name / f"seed_{cfg.seed}"
out_dir.mkdir(parents=True, exist_ok=True)
ts = datetime.now(ZoneInfo('America/New_York')).strftime("%Y%m%d_%H%M%S")
save_path = out_dir / f"checkpoints_{cfg.concept_name}_seed_{cfg.seed}_{ts}.pt"
torch.save(checkpoints, save_path)

print(f"\n✅ 训练完成!")
print(f"💾 检查点: {save_path}")

## Cell 6: 保存到 Google Drive (可选)

In [ ]:
from google.colab import drive
import shutil
from pathlib import Path

drive.mount('/content/drive')

save_dir = Path("/content/drive/MyDrive/MyVLM_art_outputs")
save_dir.mkdir(parents=True, exist_ok=True)

shutil.copytree("/content/MyVLM_art/outputs", save_dir / "outputs", dirs_exist_ok=True)
print(f"✅ 已保存到: {save_dir / 'outputs'}")